## Procedural Terrain Generation -- Perlin Noise

### Details

- The CPU code is implemented in Numba and parallelized with the `@jit` flag, `parallel=True`. It has been optimized and operates on single precision floating point numbers (`np.float32`).

- The Dask CPU code uses dask arrays and a call to `dask.array.map_blocks()` to distribute the computation. The function run locally by each process is the same as the plain Numba implementation. It also uses `np.float32` numbers. 

- The GPU code is implemented in Numba and Cupy. Optimizations performed:
  - Uses a thread local array to store a 4,2 array used in gradient function
  - Each thread calculated multiple output elements. This way the code can handle abstractly large input dimensions, and runs more efficiently.
  - The block and grid dimensions have been fine tuned for the V100 GPU.
  - Uses `float32` numbers. 

In [ ]:
import os
import numpy as np
import datashader as ds
import xarray as xr

from xrspatial import fast_perlin

In [ ]:
# This just an example input size. Generally, the GPU speedup is related to the input size.
W = 1920
H = 1080
freq = (2., 3.)
seed = 0


### Load Data

In [ ]:
# Numpy + numba

# Use float32 datatype
numpy_noise = xr.DataArray(np.zeros((H, W), dtype=np.float32),
        name='numpy_terrain',
        dims=('y', 'x'),
        attrs={'res': 1})



In [ ]:
# Transfer terrain to the GPU
import cupy
cupy_noise = xr.DataArray(cupy.zeros((H, W), dtype=np.float32),
                name='cupy_terrain',
                dims=('y', 'x'),
                attrs={'res': 1})


In [ ]:
# Setup dask cluster
from dask.distributed import Client
client = Client(processes=False, threads_per_worker=1, n_workers=4, memory_limit='2GB')

In [ ]:
# Dask terrain
import dask.array as da
dask_numpy_noise = xr.DataArray(da.zeros((H, W), dtype=np.float32, chunks=2048),
                name='dask_numpy_terrain',
                dims=('y', 'x'),
                attrs={'res': 1})
dask_numpy_noise.persist()

### CPU Benchmarking

- 4xLarge 16 Cores 128GB RAM

In [ ]:
# run cpu benchmark
cpu_time = %timeit -o cpu_res = fast_perlin(numpy_noise, freq, seed)

### Dask Benchmarking

In [ ]:
# run dask benchmark
dask_cpu_time = %timeit -o dask_cpu_res = fast_perlin(dask_numpy_noise, freq, seed).compute()

### GPU Benchmarking

- T4
- V100

In [ ]:
# run gpu benchmark
gpu_time = %timeit -o gpu_res = fast_perlin(cupy_noise, freq, seed)

### Calculate and report the results. 

In [ ]:
# CPU time  
mean_cpu_time = np.mean(cpu_time.all_runs)/cpu_time.loops
std_cpu_time = np.std(cpu_time.all_runs)/cpu_time.loops

# necessary initializations
mean_dask_cpu_time = std_dask_cpu_time = 0
mean_gpu_time = std_gpu_time = 0
speedup_dask = speedup_gpu = 0

In [ ]:
# DASK time and speedup
mean_dask_cpu_time = np.mean(dask_cpu_time.all_runs)/dask_cpu_time.loops
std_dask_cpu_time = np.std(dask_cpu_time.all_runs)/dask_cpu_time.loops

speedup_dask = mean_cpu_time / mean_dask_cpu_time


In [ ]:
# GPU time and speedup
mean_gpu_time = np.mean(gpu_time.all_runs)/gpu_time.loops
std_gpu_time = np.std(gpu_time.all_runs)/gpu_time.loops

speedup_gpu = mean_cpu_time / mean_gpu_time


In [ ]:
print('HxW        CPU Time (sec)    Dask Time (sec)    Speedup Dask    GPU Time (sec)    Speedup GPU')
print('{}x{}   {:.3f} ± {:.3f}    {:.3f} ± {:.3f}    {:.2f}x    {:.3f} ± {:.3f}    {:.2f}x'.format(
        H, W, mean_cpu_time, std_cpu_time,
        mean_dask_cpu_time, std_dask_cpu_time, speedup_dask,
        mean_gpu_time, std_gpu_time, speedup_gpu))

In [ ]:
# stop the dask cluster
client.close()